In [1]:
import os
from dotenv import load_dotenv

os.chdir("..")
load_dotenv()

from app.utils.data_ingestion_utils import (
    ingest_stocks_eod,
    ingest_stocks_insider_trades,
    ingest_stocks_metadata,
    ingest_stocks_fundamental_income_statement,
    ingest_stocks_fundamental_balance_sheet,
    ingest_stocks_fundamental_cash_flow,
    ingest_stocks_fundamental_earnings_estimates,
)


In [2]:
companies = ["IBM", "GOOG", "META", "ASML", "NVDA", "MSFT", "AAPL"]

for company in companies:
    stocks_eod_response = ingest_stocks_eod(company)
    print(f"Ingestion complete stocks EOD for {company}, errors: {stocks_eod_response.json().get('errors')}")

Ingestion complete stocks EOD for IBM, errors: False
Ingestion complete stocks EOD for GOOG, errors: False
Ingestion complete stocks EOD for META, errors: False
Ingestion complete stocks EOD for ASML, errors: False
Ingestion complete stocks EOD for NVDA, errors: False
Ingestion complete stocks EOD for MSFT, errors: False
Ingestion complete stocks EOD for AAPL, errors: False


In [ ]:
stocks_insider_trades_response = ingest_stocks_insider_trades("IBM")
print(f"Stocks insider trades ingestion complete with errors: {stocks_insider_trades_response.json().get('errors')}")

In [3]:
stocks_metadata_response = ingest_stocks_metadata("IBM")
print(f"Stocks metadata ingestion complete with errors: {stocks_metadata_response.json().get('errors')}")

Stocks metadata ingestion complete with errors: False


In [4]:
stocks_fundamental_income_statement_response = ingest_stocks_fundamental_income_statement("IBM")
print(f"Stocks fundamental income statement ingestion complete with errors: {stocks_fundamental_income_statement_response.json().get('errors')}")

Stocks fundamental income statement ingestion complete with errors: False


In [5]:
stocks_fundamental_balance_sheet_response = ingest_stocks_fundamental_balance_sheet("IBM")
print(f"Stocks fundamental balance sheet ingestion complete with errors: {stocks_fundamental_balance_sheet_response.json().get('errors')}")

Stocks fundamental balance sheet ingestion complete with errors: False


In [6]:
stocks_fundamental_cash_flow_response = ingest_stocks_fundamental_cash_flow("IBM")
print(f"Stocks fundamental cash flow ingestion complete with errors: {stocks_fundamental_cash_flow_response.json().get('errors')}")

Stocks fundamental cash flow ingestion complete with errors: False


In [7]:
stocks_fundamental_earnings_estimates_response = ingest_stocks_fundamental_earnings_estimates("IBM")
print(f"Stocks fundamental estimated earnings ingestion complete with errors: {stocks_fundamental_earnings_estimates_response.json().get('errors')}")

Stocks fundamental estimated earnings ingestion complete with errors: False
